In [2]:
df = spark.read.option("multiline", "true").json("Files/latest_news.json")
display(df)

StatementMeta(, 73732a30-04b1-4a83-8095-ef6e69a917dd, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a013c1b5-adfa-4d43-b017-90ee6748efb5)

In [3]:
df = df.select("data")
display(df)

StatementMeta(, 73732a30-04b1-4a83-8095-ef6e69a917dd, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, ddee2e53-b874-446d-96fb-b82989c86619)

In [4]:
from pyspark.sql.functions import explode
df_exploded = df.select(explode(df["data"]).alias('json_object'))
display(df_exploded)

StatementMeta(, 73732a30-04b1-4a83-8095-ef6e69a917dd, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, b501233d-48ed-49b0-bca5-2f5b2ab61e0e)

In [5]:
json_news = df_exploded.toJSON().collect()

StatementMeta(, 73732a30-04b1-4a83-8095-ef6e69a917dd, 7, Finished, Available, Finished)

In [6]:
import json

news_dict = json.loads(json_news[0])
print(news_dict['json_object']['description'])

StatementMeta(, 73732a30-04b1-4a83-8095-ef6e69a917dd, 8, Finished, Available, Finished)

In [7]:
# we need description, image_url, title, url, categories, keywords, published_at, source

print(news_dict['json_object']['title'])
print(news_dict['json_object']['description'])
print(news_dict['json_object']['image_url'])
print(news_dict['json_object']['source'])
print(news_dict['json_object']['categories'])
print(news_dict['json_object']['keywords'])
print(news_dict['json_object']['published_at'])
print(news_dict['json_object']['url'])

StatementMeta(, 73732a30-04b1-4a83-8095-ef6e69a917dd, 9, Finished, Available, Finished)

People act as key links in Sino-U.S. relationship

https://poss-videocloud.cns.com.cn/oss/2024/07/01/chinanews/MEIZI_YUNSHI/onair/F45E7BABA7474A80B1DA17A1D9C4CAC8.jpg
ecns.cn
['general']

2024-07-02T08:19:38.000000Z
http://www.ecns.cn/news/politics/2024-07-02/detail-ihecxmxh6559751.shtml


In [8]:
title = []
description = []
image_url = []
source = []
categories = []
keywords = []
published_at = []
url = []

for json_str in json_news:
    try:
        article = json.loads(json_str)

        title.append(article['json_object']['title'])
        description.append(article['json_object']['description'])
        image_url.append(article['json_object']['image_url'])
        source.append(article['json_object']['source'])
        categories.append(article['json_object']['categories'])
        keywords.append(article['json_object']['keywords'])
        published_at.append(article['json_object']['published_at'])
        url.append(article['json_object']['url'])

    except Exception as e:
        print(f"error: {e}")


print(title)

StatementMeta(, 73732a30-04b1-4a83-8095-ef6e69a917dd, 10, Finished, Available, Finished)

['People act as key links in Sino-U.S. relationship', 'New Yankee 14 Through the History Mirror Collectors Edition-RAZOR – ReleaseBB', 'Thai Constitutional Court to Conclude Key Political Cases By September']


In [9]:
from pyspark.sql.types import StructType,StructField,StringType

data = list(zip(title,description,image_url,source,categories,keywords,published_at,url))

schema = StructType([
    StructField('title',StringType(),True),
    StructField('description',StringType(),True),
    StructField('image_url',StringType(),True),
    StructField('source',StringType(),True),
    StructField('categories',StringType(),True),
    StructField('keywords',StringType(),True),
    StructField('published_at',StringType(),True),
    StructField('url',StringType(),True)
])

final_df = spark.createDataFrame(data,schema=schema)

display(final_df)

StatementMeta(, 73732a30-04b1-4a83-8095-ef6e69a917dd, 11, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 1f7b3e8c-3e3a-4092-af81-ba7fe00de23f)

In [10]:
from pyspark.sql.functions import to_date, date_format

final_df = final_df.withColumn("published_at", date_format(to_date("published_at"), "dd-MMM-yyyy"))

display(final_df)

StatementMeta(, 73732a30-04b1-4a83-8095-ef6e69a917dd, 12, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c7806078-9569-42cd-b930-e32874ff9313)

In [16]:
from pyspark.sql.utils import AnalysisException

try:

    table_name = 'news_lake.news_table'

    final_df.write.format('delta').saveAsTable(table_name)

except:

    print('Table exists! Updating...')

    final_df.createOrReplaceTempView('vw_final_df')

    spark.sql(f""" MERGE INTO {table_name} target_table
                   USING vw_final_df source_view

                   ON source_view.url = target_table.url

                   WHEN MATCHED AND

                   source_view.title <> target_table.title OR
                   source_view.description <> target_table.description OR
                   source_view.image_url <> target_table.image_url OR
                   source_view.source <> target_table.source OR
                   source_view.categories <> target_table.categories OR
                   source_view.keywords <> target_table.keywords OR
                   source_view.published_at <> target_table.published_at

                   THEN UPDATE SET *

                   WHEN NOT MATCHED THEN INSERT *
    
                    """)

StatementMeta(, 73732a30-04b1-4a83-8095-ef6e69a917dd, 18, Finished, Available, Finished)

table exists...
